# Demo hotelling

Illustrate the Hotelling Transform

## Image creation

A binary image with an ellipse

In [4]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import numpy.linalg as LA
import sys,os
ia898path = '../../'
if ia898path not in sys.path:
    sys.path.append(ia898path)
import ia898.src as ia

In [3]:
f = ia.gaussian((100,100), np.array([[45,50]]).T, [[40*40,10*10],[10*10,20*20]])
f = f > (f.max() * np.exp(-1./2))
ia.adshow(f, title='Ellipse')     

AttributeError: module 'ia898.src' has no attribute 'gaussian'

## Feature vector

The coordinates of each 1 pixel are the features: cols and rows coordinates. 

In [ ]:
X = np.nonzero(f)
X = np.array(X).T
n,m = X.shape
print('Number of samples (n):', n)
print('Number of attributes (m):', m)
print('X=\n', end=' ')
print(X)

## Mean vector

The mean value is the centroid.

$$   \overline{X} = \frac{1}{n} [ \sum_{i = 0}^{n-1} X_{i1}   \sum_{i = 0}^{n-1} X_{i2}   \cdots  
   \sum_{i = 0}^{n-1} X_{im} ]
$$

In [ ]:
mu = X.mean(axis=0)
print('Centroid = ',mu)

## Computing the covariance matrix

The covariance matrix is given by:

$$
   \Sigma = \frac{1}{n-1}(X - \overline{X})^T(X - \overline{X})
$$

It can be computed by the numpy function cov or directly from the equation:

In [ ]:
C = np.cov(X-mu, rowvar=False)
C1 = ((X-mu).T).dot(X-mu)/(n-1)
print('Covariance matrix computed usint np.cov\n',C.round(2))
print('Covariance matrix computed from the definition\n',C1.round(2))

## Computing eigenvalues and eigenvectors

The eigenvalues and eigenvectors are computed from the covariance matrix. 
The eigenvalues are sorted in decrescent order

In [ ]:
[e,V] = LA.eigh(C)
print('e:',e.round(2))
print('V:\n',V.round(2))
index = np.argsort(e)[::-1]
V = V[:,index]
e = e[index]
print('\nEigenvalues vector\n')
print(e.round(2))
print('\nEigenvectors matrix\n')
print(V.round(2))
print('\nInverse is transpose\n', end=' ')
print(LA.inv(V).round(2))

## Measure the angle of inclination

The direction of the eigenvector of the largest eigenvalue gives the inclination of the elongated figure

In [ ]:
#sigma = np.sqrt(e)
theta = arctan(V[0,0]/V[1,0])*180./np.pi
print('The inclination angle is', theta.round(3),' degrees')

## Plot the eigenvectors and centroid

The eigenvectors are placed at the centroid and scaled by the square root of its correspondent eigenvalue

In [ ]:
ia.adshow(f, title='Ellipse')

## Compute the Hotelling transform

The Hotelling transform, also called Karhunen-Loeve (K-L) transform, or the method of principal components, is computed below

In [ ]:
Xnew = (X-mu).dot(V)
Xnew_mu = Xnew.mean(axis=0)
print('Mean value of the transform\n')
print(Xnew_mu.round(2))
C_Xnew = np.cov(Xnew,rowvar=False)
print('\nCovariance matrix of the transform\n')
print(C_Xnew.round(2))
C_Xnew = np.where(C_Xnew <1e-10, 0, C_Xnew)
print('\nStandard deviation matrix of the transformed data\n')
print(np.sqrt(C_Xnew).round(2))

## Display the transformed data

The centroid of the transformed data is zero (0,0). To visualize it as an image, the features are 
translated by the centroid of the original data, so that only the rotation effect of the Hotelling 
transform is visualized.

The black dots in the transformed image is due to the direct geometric transform.

In [ ]:
Xnew = np.rint(Xnew + mu).astype(int)
g = np.zeros(f.shape, np.bool)
g[Xnew[:,1],Xnew[:,0]] = True
ia.adshow(g, title='Ellipse w/ straightened axes')

## Image read and display

The RGB color image is read and displayed

In [ ]:
f = adread('boat.ppm')
adshow(f,title='Original RGB image')

## Extracting and display RGB components

The color components are stored in the third dimension of the image array

In [ ]:
r = f[0,:,:]
g = f[1,:,:]
b = f[2,:,:]
ia.adshow(r,title='Red component')
ia.adshow(g,title='Green component')
ia.adshow(b,title='Blue component')

## Feature vector: R, G and B values

The features are the red, green and blue components. The mean vector is the average color in the image. The eigenvalues and eigenvectors are computed. The dimension of the covariance matrix is 3x3 as there are 3 features in use

In [ ]:
X = np.array([r.ravel(),g.ravel(),b.ravel()]).T
print(X.shape)
V,e,mu = ia.iapca(X)
print('Mean value of each channel (R,G,B)\n')
print(mu.round(2))
Cx = np.cov(X,rowvar=False)
print('\nCovariance matrix\n')
print(Cx.round(2))
print('\nEigenvalues matrix\n')
print(e.round(2))
print('\nEigenvectors matrix\n')
print(V.round(2))

## Hotelling transform

The K-L transform is computed. The mean vector is zero and the covariance matrix is decorrelated. We can see the values of the standard deviation of the first, second and third components

In [ ]:
y = (X-mu).dot(V)
my = mean(y,axis=0)
print('Mean value of the transform\n')
print(my.round(3))
Cy = np.cov(y,rowvar=False)
print('\nCovariance matrix of the transform\n')
print(Cy.round(2))
Cy = np.where(Cy <1e-10, 0, Cy)
print('\nStandard deviation matrix of the transformed data\n')
print(np.sqrt(Cy).round(2))

## Displaying each component of the transformed image

The transformed features are put back in three different images with the g1 with the first component (larger variance) and g3 with the smaller variance

In [ ]:
g1 = y[:,0].reshape(r.shape)
g2 = y[:,1].reshape(r.shape)
g3 = y[:,2].reshape(r.shape)
ia.adshow(255-ia.ianormalize(g1),title='Variance:%f' % (Cy[0,0],))
ia.adshow(ia.ianormalize(g2),title='Variance:%f' % (Cy[1,1],))
ia.adshow(ia.ianormalize(g3),title='Variance:%f' % (Cy[2,2],))
    

## Full reconstruction

One can reconstruct the image by multiplying the transformed image by the transpose of eigenvector matrix  
and adding the mean value of the image.

$$ \begin{matrix}
   Y &=& (X - \bar{X}) V \\
   X &=& \bar{X} + Y V^{T}
   \end{matrix}
   $$
   

In [ ]:
y1=y.copy()
X1 = y1.dot(V.T) + mu
f1=np.zeros_like(f)            
for i in range(3):
    f1[i,:,:]=reshape(X1[:,i],r.shape)
print('Mean square error = ',np.mean((f1-f)**2).round(2))
print('Max square error = ',np.max((f1-f)**2).round(2))
adshow(f,title='original')
adshow(f1,title='Recomposed image')

## Partial reconstruction

The component corresponding to the smallest variance is erased from the transformed image.
It is noticeable that the mean square error is the smallest eigenvalue.

In [ ]:
y1=y.copy()
y1[:,2]=0
X1 = y1.dot(V.T) + mu
print('iaimginfo(X1):', ia.iaimginfo(X1))
X1n = ia.ianormalize(X1).astype('uint8')
f1=np.zeros_like(f)            
for i in range(3):
    f1[i,:,:]=reshape(X1n[:,i],r.shape)
print('Mean square error = ',np.mean((1.*f1-f)**2).round(2))
print('Max square error = ',np.max((1.*f1-f)**2).round(2))
adshow(f,title='original')
adshow(f1,title='Recomposed image')
adshow(ia.ianormalize(np.abs(1*f1-f)),'normalized absolute error')

## References

- `http://en.wikipedia.org/wiki/Karhunen%E2%80%93Lo%C3%A8ve_theorem Karhunen Loeve Transform` - Wikipedia
- `http://www.visiondummy.com/2014/04/geometric-interpretation-covariance-matrix/ Computer Vision for Dummies`: Geometric
   interpretation of the covariance matrix

## See Also

- `iapca` - Principal Component Analysis
- `iagaussian` - Gaussian image